# 🏠 House Price Prediction Lab (Starter)
Fill in the TODOs. When it works, copy the relevant code into `src/` files as instructed.


In [ ]:
# TODO: Imports (pandas, numpy, sklearn, joblib)


In [ ]:
# TODO: Load dataset from data/housing.csv (or your provided CSV)


In [ ]:
# TODO: Handle missing in categorical columns by treating 'Missing' as a category


In [ ]:
# TODO: Build ColumnTransformer
# - sqrt transform on 'sqft'
# - OneHotEncoder for ['locality','parking'] with handle_unknown='ignore'
# - passthrough for 'bedrooms'


In [ ]:
# TODO: Build Pipeline(preprocessor -> LinearRegression) and train_test_split


In [ ]:
# TODO: Evaluate using RMSE (and optional R^2)


In [ ]:
# TODO: Save model with joblib.dump(model, 'model.pkl')
